In [20]:
#welcome to Jeffery's world! Shoot me an email if you want to know more @ meik@purdue.edu
import pandas as pd
import numpy as np
import sklearn
import scipy
from sklearn.metrics import cohen_kappa_score, make_scorer
import gc

In [21]:
#load data into files
dataset = pd.read_excel('data original.xlsx')
dataset.drop(['Date', 'Comment','Authority?'],inplace=True,axis=1)
dataset.columns = ['entry','text','tagger','fraud','policy','unauthorized','fee']

dataset[['fraud','policy','unauthorized','fee']] = dataset[['fraud','policy','unauthorized','fee']].replace(['Yes','No'],[1,0])
#create master tag file
protaggers = ['lchen@vt.edu', 'jeffrey@vt.edu', 'lanham@vt.edu', 'shiva@vt.edu']

pro_tag_file = dataset.loc[dataset['tagger'].isin(protaggers)]
pro_tag_file.columns = ['entry','text','ptagger','pfraud','ppolicy','pauthorization','pfee']

tag_file = dataset.loc[~dataset['tagger'].isin(protaggers)]

#create the text file for it saves some ram....
text_file = dataset[['entry','text']]
text_file = text_file.drop_duplicates()

taggerlist = set(tag_file['tagger'])

In [22]:

policy_tagger_dataframe_kappa = pd.DataFrame()
authorization_tagger_dataframe_kappa = pd.DataFrame()
fee_tagger_dataframe_kappa = pd.DataFrame()


In [23]:
#this part is for fraud only
#create good tagger list and another dataframe with their kappa
fraud_tagger_dataframe_kappa = pd.DataFrame()
fraud_goodtagger = []

#note: there is no kappa score provided by professor. So I am the authority :D
for tagger in taggerlist:
    temp_tagged_file = tag_file.loc[tag_file['tagger']==tagger]

    comparefile = pd.merge(temp_tagged_file, pro_tag_file, how='inner', on=['entry'])

    kappa = sklearn.metrics.cohen_kappa_score(comparefile['fraud'],comparefile['pfraud'])
    if kappa >= 0.4:
        fraud_goodtagger.append(tagger)
        temp_tagger = [[tagger,kappa]]
        fraud_tagger_dataframe_kappa = fraud_tagger_dataframe_kappa.append(temp_tagger)
    gc.collect

#write the kappa of taggers to file
fraud_tagger_dataframe_kappa.columns = ['tagger','kappa']
print('fraud good taggers count: ',len(fraud_tagger_dataframe_kappa))
writer = pd.ExcelWriter('fraud_tagger_dataframe_kappa.xlsx')
fraud_tagger_dataframe_kappa.to_excel(writer)
writer.save()

fraudfile = dataset.loc[dataset['tagger'].isin(fraud_goodtagger)]
fraudfile.drop(['policy','unauthorized','fee'],inplace=1,axis=1)
print('fraudfile.shape: ', fraudfile.shape)

#merge fraud dataset by first
fraudfile_analysis = pd.merge(fraudfile,fraud_tagger_dataframe_kappa,on='tagger',how='left')
fraudfile_analysis.sort(['entry','kappa'],ascending=[True,False])
fraudfile_analysis_first_sort = fraudfile_analysis.groupby('entry').first()
fraudfile_analysis_first_sort.drop('kappa',inplace=1,axis = 1)
print('fraudfile.shape: ', fraudfile_analysis_first_sort.shape)

#write first-selected doc to file
writer = pd.ExcelWriter('fraudfile_analysis_first_sort.xlsx')
fraudfile_analysis_first_sort.to_excel(writer)
writer.save()


#merge fraud dataset by voting (average)
temp = fraudfile_analysis.groupby('entry', as_index=False)['fraud'].mean()
temp.columns=['entry','score']
temp['fraud'] = np.where(temp['score']>=0.5, 1,0)
fraudfile_analysis_mean_merge = pd.merge(temp,text_file,on='entry',how='left')
fraudfile_analysis_mean_merge.drop('score',axis=1,inplace=True)

#writed average-selected doc to file
print('fraudfile.shape: ', fraudfile_analysis_mean_merge.shape)
writer = pd.ExcelWriter('fraudfile_analysis_mean_merge.xlsx')
fraudfile_analysis_mean_merge.to_excel(writer)
writer.save()

fraud good taggers count:  31
fraudfile.shape:  (4670, 4)
fraudfile.shape:  (3060, 3)


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:32: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


fraudfile.shape:  (3060, 3)


In [24]:
#this part is for policy only
#create good tagger list and another dataframe with their kappa
policy_tagger_dataframe_kappa = pd.DataFrame()
policy_goodtagger = []

for tagger in taggerlist:
    temp_tagged_file = tag_file.loc[tag_file['tagger']==tagger]

    comparefile = pd.merge(temp_tagged_file, pro_tag_file, how='inner', on=['entry'])

    kappa = sklearn.metrics.cohen_kappa_score(comparefile['policy'],comparefile['ppolicy'])
    if kappa >= 0.4:
        policy_goodtagger.append(tagger)
        temp_tagger = [[tagger,kappa]]
        policy_tagger_dataframe_kappa = policy_tagger_dataframe_kappa.append(temp_tagger)
    gc.collect

#write the kappa of taggers to file
policy_tagger_dataframe_kappa.columns = ['tagger','kappa']
print('policy good taggers count: ',len(policy_tagger_dataframe_kappa))
writer = pd.ExcelWriter('policy_tagger_dataframe_kappa.xlsx')
policy_tagger_dataframe_kappa.to_excel(writer)
writer.save()

policyfile = dataset.loc[dataset['tagger'].isin(policy_goodtagger)]
policyfile.drop(['fraud','unauthorized','fee'],inplace=1,axis=1)
print('policyfile.shape: ', policyfile.shape)

#merge policy dataset by first
policyfile_analysis = pd.merge(policyfile,policy_tagger_dataframe_kappa,on='tagger',how='left')
policyfile_analysis.sort(['entry','kappa'],ascending=[True,False])
policyfile_analysis_first_sort = policyfile_analysis.groupby('entry').first()
policyfile_analysis_first_sort.drop('kappa',inplace=1,axis = 1)
print('policyfile.shape: ', policyfile_analysis_first_sort.shape)

#write first-selected doc to file
writer = pd.ExcelWriter('policyfile_analysis_first_sort.xlsx')
policyfile_analysis_first_sort.to_excel(writer)
writer.save()


#merge policy dataset by voting (average)
temp = policyfile_analysis.groupby('entry', as_index=False)['policy'].mean()
temp.columns=['entry','score']
temp['policy'] = np.where(temp['score']>=0.5, 1,0)
policyfile_analysis_mean_merge = pd.merge(temp,text_file,on='entry',how='left')
policyfile_analysis_mean_merge.drop('score',axis=1,inplace=True)

#writed average-selected doc to file
print('policyfile.shape: ', policyfile_analysis_mean_merge.shape)
writer = pd.ExcelWriter('policyfile_analysis_mean_merge.xlsx')
policyfile_analysis_mean_merge.to_excel(writer)
writer.save()


policy good taggers count:  23
policyfile.shape:  (3400, 4)
policyfile.shape:  (2408, 3)


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:31: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


policyfile.shape:  (2408, 3)


In [25]:
#this part is for unauthorized only
#create good tagger list and another dataframe with their kappa
unauthorized_tagger_dataframe_kappa = pd.DataFrame()
unauthorized_goodtagger = []

#note: there is no kappa score provided by professor. So I am the authority :D
for tagger in taggerlist:
    temp_tagged_file = tag_file.loc[tag_file['tagger']==tagger]

    comparefile = pd.merge(temp_tagged_file, pro_tag_file, how='inner', on=['entry'])

    kappa = sklearn.metrics.cohen_kappa_score(comparefile['unauthorized'],comparefile['punauthorized'])
    if kappa >= 0.4:
        unauthorized_goodtagger.append(tagger)
        temp_tagger = [[tagger,kappa]]
        unauthorized_tagger_dataframe_kappa = unauthorized_tagger_dataframe_kappa.append(temp_tagger)
    gc.collect

#write the kappa of taggers to file
unauthorized_tagger_dataframe_kappa.columns = ['tagger','kappa']
print('unauthorized good taggers count: ',len(unauthorized_tagger_dataframe_kappa))
writer = pd.ExcelWriter('unauthorized_tagger_dataframe_kappa.xlsx')
unauthorized_tagger_dataframe_kappa.to_excel(writer)
writer.save()

unauthorizedfile = dataset.loc[dataset['tagger'].isin(unauthorized_goodtagger)]
unauthorizedfile.drop(['policy','fraud','fee'],inplace=1,axis=1)
print('unauthorizedfile.shape: ', unauthorizedfile.shape)

#merge unauthorized dataset by first
unauthorizedfile_analysis = pd.merge(unauthorizedfile,unauthorized_tagger_dataframe_kappa,on='tagger',how='left')
unauthorizedfile_analysis.sort(['entry','kappa'],ascending=[True,False])
unauthorizedfile_analysis_first_sort = unauthorizedfile_analysis.groupby('entry').first()
unauthorizedfile_analysis_first_sort.drop('kappa',inplace=1,axis = 1)
print('unauthorizedfile.shape: ', unauthorizedfile_analysis_first_sort.shape)

#write first-selected doc to file
writer = pd.ExcelWriter('unauthorizedfile_analysis_first_sort.xlsx')
unauthorizedfile_analysis_first_sort.to_excel(writer)
writer.save()


#merge unauthorized dataset by voting (average)
temp = unauthorizedfile_analysis.groupby('entry', as_index=False)['unauthorized'].mean()
temp.columns=['entry','score']
temp['unauthorized'] = np.where(temp['score']>=0.5, 1,0)
unauthorizedfile_analysis_mean_merge = pd.merge(temp,text_file,on='entry',how='left')
unauthorizedfile_analysis_mean_merge.drop('score',axis=1,inplace=True)

#writed average-selected doc to file
print('unauthorizedfile.shape: ', unauthorizedfile_analysis_mean_merge.shape)
writer = pd.ExcelWriter('unauthorizedfile_analysis_mean_merge.xlsx')
unauthorizedfile_analysis_mean_merge.to_excel(writer)
writer.save()


KeyError: 'punauthorized'

In [ ]:
#this part is for fee only
#create good tagger list and another dataframe with their kappa
fee_tagger_dataframe_kappa = pd.DataFrame()
fee_goodtagger = []

#note: there is no kappa score provided by professor. So I am the authority :D
for tagger in taggerlist:
    temp_tagged_file = tag_file.loc[tag_file['tagger']==tagger]

    comparefile = pd.merge(temp_tagged_file, pro_tag_file, how='inner', on=['entry'])

    kappa = sklearn.metrics.cohen_kappa_score(comparefile['fee'],comparefile['pfee'])
    if kappa >= 0.4:
        fee_goodtagger.append(tagger)
        temp_tagger = [[tagger,kappa]]
        fee_tagger_dataframe_kappa = fee_tagger_dataframe_kappa.append(temp_tagger)
    gc.collect

#write the kappa of taggers to file
fee_tagger_dataframe_kappa.columns = ['tagger','kappa']
print('fee good taggers count: ',len(fee_tagger_dataframe_kappa))
writer = pd.ExcelWriter('fee_tagger_dataframe_kappa.xlsx')
fee_tagger_dataframe_kappa.to_excel(writer)
writer.save()

feefile = dataset.loc[dataset['tagger'].isin(fee_goodtagger)]
feefile.drop(['policy','unauthorized','fraud'],inplace=1,axis=1)
print('feefile.shape: ', feefile.shape)

#merge fee dataset by first
feefile_analysis = pd.merge(feefile,fee_tagger_dataframe_kappa,on='tagger',how='left')
feefile_analysis.sort(['entry','kappa'],ascending=[True,False])
feefile_analysis_first_sort = feefile_analysis.groupby('entry').first()
feefile_analysis_first_sort.drop('kappa',inplace=1,axis = 1)
print('feefile.shape: ', feefile_analysis_first_sort.shape)

#write first-selected doc to file
writer = pd.ExcelWriter('feefile_analysis_first_sort.xlsx')
feefile_analysis_first_sort.to_excel(writer)
writer.save()


#merge fee dataset by voting (average)
temp = feefile_analysis.groupby('entry', as_index=False)['fee'].mean()
temp.columns=['entry','score']
temp['fee'] = np.where(temp['score']>=0.5, 1,0)
feefile_analysis_mean_merge = pd.merge(temp,text_file,on='entry',how='left')
feefile_analysis_mean_merge.drop('score',axis=1,inplace=True)

#writed average-selected doc to file
print('feefile.shape: ', feefile_analysis_mean_merge.shape)
writer = pd.ExcelWriter('feefile_analysis_mean_merge.xlsx')
feefile_analysis_mean_merge.to_excel(writer)
writer.save()


In [ ]:
fraud_tagger_dataframe_kappa.head(5)

In [ ]:
fraudfile_analysis.head(5)

In [ ]:
#expereiment field
tag_file.head(5)

In [ ]:
text_file.head(5)